In [72]:
#import the package
import pandas as pd
import plotly, dash
import plotly.express as px
from dash import Dash, html, dash_table, dcc, callback, Output, Input, State

In [73]:
df = pd.read_csv('data_file.csv')
df.drop(columns = "id",inplace = True)

In [95]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21089 entries, 0 to 21088
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   timestamp  21089 non-null  object 
 1   x          21089 non-null  float64
 2   y          21089 non-null  float64
 3   z          21089 non-null  float64
dtypes: float64(3), object(1)
memory usage: 659.2+ KB


In [96]:

# Initialize the Dash app
app = dash.Dash(__name__)

# Set the initial value for the number of samples to display
initial_value = 1000

# Define the layout of the app
app.layout = html.Div([
    html.H1("Data Visualization"),  # Title of the app
    
    # Dropdown for selecting the type of chart
    html.Div([
        dcc.Dropdown(
            id='graph-dropdown',
            placeholder='Please select types of chart',  # Placeholder text
            options=[
                {'label': 'Scatter Plot', 'value': 'scatter'},
                {'label': 'Line Chart', 'value': 'line'},
                {'label': 'Distribution Plot', 'value': 'dist'}
            ],
            value='scatter'  # Default value
        )
    ], style={"width": "30%"}),  # Set width for the dropdown container
    
    # Checklist for selecting variables (X, Y, Z)
    html.Div([
        dcc.Checklist(
            id='var-list',
            options=[
                {'label': 'X', 'value': 'x'},
                {'label': 'Y', 'value': 'y'},
                {'label': 'Z', 'value': 'z'}
            ],
            value=['x'],  # Default value (X is selected by default)
            inline=True  # Display options inline
        )
    ]),
    
    # Input field for specifying the number of samples to display
    html.Div([
        dcc.Input(
            id='sample-input',
            type='number',
            value=initial_value,  # Set the initial value to 1000
            style={'margin-top': '30px'}  # Add some space above the input field
        )
    ]),
    
    # Buttons for navigating through the data (Previous and Next)
    html.Div([
        html.Button('Previous', id='previous-button', n_clicks=0),  # Previous button
        html.Button('Next', id='next-button', n_clicks=0),  # Next button
    ], style={'margin-top': '20px'}),  # Add some space above the buttons
        
    html.Div([
        html.H3(id = 'notification-area',
        style={'color': 'red', 'margin-top': '20px'})
    ]),
    # Graph component to display the selected chart
    html.Div([
        dcc.Graph(id='graph')
    ]),
    
    # Table to display summary statistics of the data
    html.Div([
        dash_table.DataTable(
            id='summary-table',
            columns=[{'name': i, 'id': i} for i in ['Statistic', 'X', 'Y', 'Z']],  # Define table columns
            style_table={'width': '50%'}  # Set width for the table
        )
    ])
])

# Callback function to update the graph and summary table based on user inputs
@app.callback(
    Output('graph', 'figure'),  # Output: Update the figure in the graph
    Output('summary-table', 'data'),  # Output: Update the data in the summary table
    Output('notification-area','children'),
    Input('graph-dropdown', 'value'),  # Input: Selected chart type
    Input('var-list', 'value'),  # Input: Selected variables (X, Y, Z)
    Input('sample-input', 'value'),  # Input: Number of samples to display
    Input('previous-button', 'n_clicks'),  # Input: Number of clicks on the Previous button
    Input('next-button', 'n_clicks')  # Input: Number of clicks on the Next button
)
def choose_graph_type(graph_dropdown, var_list, n_samples, prev_clicks, next_clicks):
    # Initialize notification message
    notification_message = ''
    
    # Validate the number of samples
    if n_samples <= 0:
        notification_message = 'Number of samples must be greater than 0.'
        # Prevent update if the sample count is invalid
        return dash.no_update, dash.no_update, notification_message
    elif n_samples >= len(df):
        notification_message = 'Number of samples must be less than total.'
        return dash.no_update, dash.no_update, notification_message
        
    # Calculate the starting index based on the number of clicks on the buttons
    total_clicks = next_clicks - prev_clicks
    start_idx = total_clicks * n_samples
    end_idx = start_idx + n_samples

    # Ensure indices are within the data range
    if start_idx < 0:
        start_idx = 0
    if end_idx > len(df):
        end_idx = len(df)
    
    # Subset the data based on the calculated indices
    data_subset = df.iloc[start_idx:end_idx]
    
    # Generate the figure based on the selected chart type
    if graph_dropdown is None:
        raise dash.exceptions.PreventUpdate()  # Prevent update if no chart type is selected
    if graph_dropdown == 'scatter':
        fig = px.scatter(data_subset, x='timestamp', y=var_list)  # Scatter plot
    elif graph_dropdown == 'line':
        fig = px.line(data_subset, x='timestamp', y=var_list)  # Line chart
    elif graph_dropdown == 'dist':
        fig = px.histogram(data_subset, x=var_list, title='Distribution Plot')  # Distribution plot
    else:
        fig = None  # Default to None if no valid chart type is selected
    
    # Calculate summary statistics (Mean, Median, Standard Deviation)
    summary_data = {
        'Statistic': ['Mean', 'Median', 'Std. Dev'],  # Statistics to calculate
        'X': [data_subset['x'].mean(), data_subset['x'].median(), data_subset['x'].std()] if 'x' in var_list else ['N/A'] * 3,
        'Y': [data_subset['y'].mean(), data_subset['y'].median(), data_subset['y'].std()] if 'y' in var_list else ['N/A'] * 3,
        'Z': [data_subset['z'].mean(), data_subset['z'].median(), data_subset['z'].std()] if 'z' in var_list else ['N/A'] * 3,
    }

    # Convert the summary statistics to a format compatible with Dash DataTable
    summary_table_data = pd.DataFrame(summary_data).to_dict('records')
    
    # Return the figure and summary table data
    return fig, summary_table_data, notification_message

# Run the Dash app
if __name__ == '__main__':
    app.run_server(jupyter_mode ='tab')

Dash app running on http://127.0.0.1:8050/


<IPython.core.display.Javascript object>